In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [5]:
MODEL_PATH = r"C:\Users\natec\OneDrive\K state Baseball Codes\Catcher Grade Project\Strike_Probability_model.pkl"
FEATURE_PATH = r"C:\Users\natec\OneDrive\K state Baseball Codes\Catcher Grade Project\Strike_Probability_features.pkl"

model = joblib.load(MODEL_PATH)
features = joblib.load(FEATURE_PATH)


In [10]:
df_game = pd.read_csv(r'C:\Users\natec\OneDrive\K state Baseball Codes\FWS 3.csv')

In [12]:
df = df_game[['PlateLocHeight', 'PlateLocSide', 'VertBreak', 'HorzBreak']]

In [13]:
df_game['StrikeProb'] = model.predict_proba(df)[:, 1]


In [14]:
df_game.head()

,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,...,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence,StrikeProb
0,1,11/8/2025,02:00.8,1,1,"Sheffield, Lincoln",830369,Left,KAN_WIL,"Lodice, Kyan",...,NaN,High,High,High,High,Undefined,NaN,NaN,NaN,0.942495
1,2,11/8/2025,02:17.0,1,2,"Sheffield, Lincoln",830369,Left,KAN_WIL,"Lodice, Kyan",...,NaN,High,High,High,Estimated,Estimated,NaN,NaN,NaN,0.719831
2,3,11/8/2025,02:43.4,1,3,"Sheffield, Lincoln",830369,Left,KAN_WIL,"Lodice, Kyan",...,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN,0.815409
3,4,11/8/2025,03:00.6,1,4,"Sheffield, Lincoln",830369,Left,KAN_WIL,"Lodice, Kyan",...,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN,0.003258
4,5,11/8/2025,03:16.5,1,5,"Sheffield, Lincoln",830369,Left,KAN_WIL,"Lodice, Kyan",...,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN,0.354584


In [22]:
df_catcher = df_game[['Catcher','PlateLocHeight', 'PlateLocSide', 'VertBreak', 'HorzBreak','PitchCall','StrikeProb']]

In [24]:
df_game['StrikeActual'] = (df_game['PitchCall'] == 'StrikeCalled').astype(int)

df_game['Framing_Score'] = 0.0

df_game.loc[
    df_game['PitchCall'].isin(['StrikeCalled', 'BallCalled']),
    'Framing_Score'
] = df_game['StrikeActual'] - df_game['StrikeProb']


In [25]:
df_game[['StrikeActual', 'StrikeProb', 'Framing_Score']].describe()

,StrikeActual,StrikeProb,Framing_Score
count,190.000000,190.000000,190.000000
mean,0.221053,0.501792,0.022675
std,0.416052,0.407936,0.256843
min,0.000000,0.000065,-0.919518
25%,0.000000,0.031142,-0.003714
50%,0.000000,0.555157,0.000000
75%,0.000000,0.952199,0.000000
max,1.000000,0.988685,0.999893


In [26]:
(
    df_game['StrikeActual'].sum(),
    df_game['StrikeProb'].sum(),
    df_game['Framing_Score'].sum()
)


(np.int64(42), np.float32(95.34056), np.float64(4.308191889234877))

In [27]:
called = df_game[df_game['PitchCall'].isin(['StrikeCalled', 'BallCalled'])]


In [29]:
catcher_framing = (
    called
    .groupby('Catcher')
    .agg(
        ExtraStrikes=('Framing_Score', 'sum'),
        ExpectedStrikes=('StrikeProb', 'sum'),
        ActualStrikes=('StrikeActual', 'sum'),
        CalledPitches=('Framing_Score', 'count')
    )
    .reset_index()
)


In [30]:
catcher_framing['ExtraStrikes_per_100'] = (
    100 * catcher_framing['ExtraStrikes'] / catcher_framing['CalledPitches']
)


In [31]:
catcher_framing['Check'] = (
    catcher_framing['ActualStrikes']
    - catcher_framing['ExpectedStrikes']
    - catcher_framing['ExtraStrikes']
)


In [32]:
catcher_framing.head()

,Catcher,ExtraStrikes,ExpectedStrikes,ActualStrikes,CalledPitches,ExtraStrikes_per_100,Check
0,"Madliak, Bear",-0.465600,22.465599,22,55,-0.846545,6.842311e-07
1,"McGahan, Shea",4.773792,15.226209,20,51,9.360376,-3.203531e-07


In [41]:
df_game['StrikeProb'] = df_game['StrikeProb'].round(2)
df_game.head()

,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,...,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence,StrikeProb,StrikeActual,Framing_Score
0,1,11/8/2025,02:00.8,1,1,"Sheffield, Lincoln",830369,Left,KAN_WIL,"Lodice, Kyan",...,High,High,High,Undefined,NaN,NaN,NaN,0.94,0,0.000000
1,2,11/8/2025,02:17.0,1,2,"Sheffield, Lincoln",830369,Left,KAN_WIL,"Lodice, Kyan",...,High,High,Estimated,Estimated,NaN,NaN,NaN,0.72,0,0.000000
2,3,11/8/2025,02:43.4,1,3,"Sheffield, Lincoln",830369,Left,KAN_WIL,"Lodice, Kyan",...,High,High,NaN,NaN,NaN,NaN,NaN,0.82,0,-0.815409
3,4,11/8/2025,03:00.6,1,4,"Sheffield, Lincoln",830369,Left,KAN_WIL,"Lodice, Kyan",...,High,High,NaN,NaN,NaN,NaN,NaN,0.00,0,-0.003258
4,5,11/8/2025,03:16.5,1,5,"Sheffield, Lincoln",830369,Left,KAN_WIL,"Lodice, Kyan",...,High,High,NaN,NaN,NaN,NaN,NaN,0.35,0,0.000000


In [43]:
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf as pdf

# Define column name mapping
column_mapping = {
    'PitchofPA': 'Pitch of PA',
    'Batter': 'Batter',
    'Inning': 'Inning',
    'Pitcher': 'Pitcher',
    'TaggedPitchType': 'Pitch Type',
    'PitchCall': 'Pitch Called',
    'RelSpeed': 'Velocity',
    'InducedVertBreak': 'IVB',
    'HorzBreak': 'HB',
    'StrikeProb': 'Strike Probability',
    'Framing_Score': 'Framing Score'
}

# Create a PDF file to save the tables
with pdf.PdfPages('Catcher Innings Tables.pdf') as pdf_pages:
    for inning in df_game['Inning'].unique():

        # Iterate through unique Top/Bottom within each inning
        for tb in df_game['Top/Bottom'].unique():

            # Get the unique catchers for this inning and Top/Bottom
            unique_catchers = df_game[
                (df_game['Inning'] == inning) &
                (df_game['Top/Bottom'] == tb)
            ]['Catcher'].unique()

            for catcher in unique_catchers:

                # Filter DataFrame for the current inning, Top/Bottom, and catcher
                inning_tb_df = df_game[
                    (df_game['Inning'] == inning) &
                    (df_game['Top/Bottom'] == tb) &
                    (df_game['Catcher'] == catcher)
                ].copy()

                # Skip if DataFrame is empty
                if inning_tb_df.empty:
                    continue

                # Create a combined column for "Tagged Hit Type" and "Play Result"
                inning_tb_df['CombinedResult'] = (
                    inning_tb_df['TaggedHitType'] + " - " + inning_tb_df['PlayResult']
                )

                # Replace "InPlay" in "Pitch Called" with values from the combined result
                inning_tb_df.loc[
                    inning_tb_df['PitchCall'] == 'InPlay',
                    'PitchCall'
                ] = inning_tb_df['CombinedResult']

                # Rename columns
                inning_tb_df_renamed = inning_tb_df.rename(columns=column_mapping)

                # Select final columns for the table
                inning_tb_table = inning_tb_df_renamed[
                    [
                        'Pitch of PA',
                        'Batter',
                        'Pitcher',
                        'Pitch Type',
                        'Pitch Called',
                        'IVB',
                        'HB',
                        'Framing Score',
                        'Strike Probability'
                    ]
                ].copy()

                # ---- DISPLAY FORMATTING (IMPORTANT FIX) ----
                format_cols = ['IVB', 'HB', 'Framing Score', 'Strike Probability']
                for col in format_cols:
                    inning_tb_table[col] = inning_tb_table[col].apply(
                        lambda x: f"{x:.2f}" if isinstance(x, (int, float)) else x
                    )
                # --------------------------------------------

                # Create a figure and axes for the table
                fig, ax = plt.subplots(figsize=(14, 10))

                # Hide axes
                ax.axis('off')

                # Add a title
                ax.set_title(
                    f"{catcher} Inning {inning} - {tb} Pitch Table",
                    fontsize=16
                )

                # Add the table to the axes
                table = ax.table(
                    cellText=inning_tb_table.values,
                    colLabels=inning_tb_table.columns,
                    loc='center',
                    cellLoc='center',
                    colLoc='center'
                )

                table.scale(1, 1.25)
                table.auto_set_font_size(False)
                table.set_fontsize(12)

                # Remove table borders
                for key, cell in table.get_celld().items():
                    cell.set_linewidth(0)
                    cell.set_edgecolor('none')

                # Auto-adjust column widths
                table.auto_set_column_width(
                    col=list(range(len(inning_tb_table.columns)))
                )

                # Make column headers bold
                for col in range(len(inning_tb_table.columns)):
                    cell = table.get_celld()[(0, col)]
                    cell.get_text().set_fontweight('bold')

                # Save page to PDF
                pdf_pages.savefig(fig)
                plt.close(fig)

print("Table has been saved")


Table has been saved


In [44]:
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf as pdf
import numpy as np
import pandas as pd

with pdf.PdfPages('Catcher_Framing_Report.pdf') as pdf_pages:

    for catcher in df_game['Catcher'].unique():

        catcher_df = df_game[df_game['Catcher'] == catcher].copy()

        # Only use called pitches
        catcher_df = catcher_df[
            catcher_df['PitchCall'].isin(['BallCalled', 'StrikeCalled'])
        ]

        if catcher_df.empty:
            continue

        # ---- Core Metrics ----
        total_pitches = len(catcher_df)
        called_strikes = catcher_df['StrikeActual'].sum()
        called_balls = total_pitches - called_strikes

        total_framing = catcher_df['Framing_Score'].sum()
        avg_strike_prob = catcher_df['StrikeProb'].mean()
        called_strike_pct = catcher_df['StrikeActual'].mean() * 100

        strikes_above_expected = total_framing
        strikes_per_100 = (strikes_above_expected / total_pitches) * 100

        # ---- Build Summary Table ----
        summary_data = {
            'Metric': [
                'Called Pitches',
                'Called Strikes',
                'Called Balls',
                'Avg Strike Probability',
                'Called Strike %',
                'Strikes Above Expected',
                'Strikes / 100'
            ],
            'Value': [
                total_pitches,
                called_strikes,
                called_balls,
                f"{avg_strike_prob:.3f}",
                f"{called_strike_pct:.1f}%",
                f"{strikes_above_expected:.2f}",
                f"{strikes_per_100:.2f}"
            ]
        }

        summary_df = pd.DataFrame(summary_data)

        # ---- Create Figure ----
        fig, ax = plt.subplots(figsize=(10, 5))
        ax.axis('off')

        ax.set_title(
            f"{catcher} — Game Framing Summary",
            fontsize=18,
            fontweight='bold'
        )

        table = ax.table(
            cellText=summary_df.values,
            colLabels=summary_df.columns,
            loc='center',
            cellLoc='center',
            colLoc='center'
        )

        table.scale(1, 2)
        table.auto_set_font_size(False)
        table.set_fontsize(12)

        # Remove borders
        for key, cell in table.get_celld().items():
            cell.set_linewidth(0)
            cell.set_edgecolor('none')

        # Bold header
        for col in range(len(summary_df.columns)):
            table.get_celld()[(0, col)].get_text().set_fontweight('bold')

        # Auto column width
        table.auto_set_column_width(col=list(range(len(summary_df.columns))))

        # Save page
        pdf_pages.savefig(fig, bbox_inches='tight')
        plt.close(fig)

print("Catcher framing report saved.")


Catcher framing report saved.


In [ ]:
import PyPDF2

def merge_pdfs(pdf_files, output_path):
    merger = PyPDF2.PdfMerger()
    for pdf_file in pdf_files:
        merger.append(pdf_file)
    merger.write(output_path)
    merger.close()

# List of PDF files you want to merge
pdf_files = ['Catcher Innings Tables.pdf', 
             'Catcher_Framing_Report.pdf', 
             ]

# Output file path for the merged PDF
output_path = r"C:\Users\JoshWintroub\OneDrive - K-State Athletics, INC\Trackman Files\Trackman Reports/FWS3 Catcher report.pdf"

# Merge the PDF files
merge_pdfs(pdf_files, output_path)

print("PDF files merged successfully!")